In [1]:
import requests
import pandas as pd
import numpy as np
import regex as re
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pandas import json_normalize
# K-means
import numpy as np
import matplotlib.pyplot as plt
import pickle

from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
top_songs = pd.read_csv('top_songs_csv')

In [3]:
list_with_all3 = pd.read_csv('list_with_all3_csv')

In [4]:
secrets_file = open("secrets.txt","r")

In [5]:
def song_recommender(title, artist):
    #Convert the input to lower case
    title_input = remove_special_characters(title.lower())
    artist_input = remove_special_characters(artist.lower())
    
    # Check if the input combination is present in the dataframe
    result = top_songs_lower[(top_songs_lower['song'].values == title_input) & (top_songs_lower['artist'].values == artist_input)]
    #print(result)

    if not result.empty:
        #Get the indices
        indices_to_remove = result.index
        temp_top_songs = top_songs.drop(indices_to_remove)

        rec_song = temp_top_songs.sample().squeeze()
        print(f"You might also like: {rec_song['song']} by {rec_song['artist']}")
    else:
        #Opening the spotify connection
        sp = open_spotify_connection()
        
        #querying spotify
        results = sp.search(q="artist:" + artist_input + " track:" + title_input, type="track", limit = 1)
        
        #First check if the query returns results (i.e total>0)
        if results['tracks']['total']!=0 :
            track_id = results['tracks']['items'][0]['id'] #Id of the first track
            #print(track_id)

            #Getting the audio features using the id
            new_data = sp.audio_features(track_id)
            #print(type(new_data[0]))
            data_df = json_normalize(new_data[0]) #normalizing the data to a df
            num_data_df = data_df.select_dtypes(include='number') #getting the numerical data only to fit
            #print(num_data_df)

            #Using pickle to get the existing scaler and kmean
            kmeans = pickle.load(open('kmean.pkl', 'rb'))
            scaler = pickle.load(open('scaler.pkl','rb'))
            X_prep = scaler.transform(num_data_df)
            predicted_cluster = kmeans.predict(X_prep)
            cluster = predicted_cluster[0]
            #print(cluster)
            cluster_songs = list_with_all3[list_with_all3['cluster'] == cluster]
            rec_song = cluster_songs[cluster_songs['cluster']== cluster].sample().squeeze()
            #print(rec_song)
            print(f"You might also like: {rec_song['song']} by {rec_song['artist']}")
        else:
            print("The song is not in Spotify. Please try again")

def remove_special_characters(text):
    # Use a regular expression to replace non-alphanumeric characters with an empty string
    return ''.join(e for e in text if e.isalnum() or e.isspace())

def open_spotify_connection():
    #Opens a new spotify connection
    secrets_file = open("secrets.txt","r")
    string = secrets_file.read()

    secrets_dict={}
    for line in string.split('\n'):
        if len(line) > 0:
            #print(line.split(':'))
            secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()
    sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))
    return sp;

In [6]:
# Getting all lower str
top_songs = pd.read_csv('top_songs_csv')
top_songs_lower = top_songs.apply(lambda x: x.astype(str).str.lower())
top_songs_lower['song'] = top_songs_lower['song'].apply(remove_special_characters)
#top_songs_lower

In [7]:
#Get user input
from pandas import json_normalize
title_input = input('Please Enter the Song Title: ')
artist_input = input('Please Enter the Artist Name: ')
song_recommender(title_input, artist_input)

Please Enter the Song Title: why can't i be you
Please Enter the Artist Name: the cure
You might also like: Taped Up Heart (feat. Clara Mae) by Clara Mae
